<a href="https://colab.research.google.com/github/shreyamaurya029/AI-vs-Human-/blob/main/RNNwithfeaturesFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Input, Concatenate,Dropout
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Classroom/traindataset.csv")

In [ ]:
df.head(2)

,Text,Label,Vocab Size,Avg Word Length,Density,active,passive,noun,pron,verb,...,det,propn,part,intj,punct,text_str,Flesch Reading Ease,Gunning Fog Index,Perplexity,Burstness
0,"['Basically there are many categories of "" Bes...",0,134,3.602230,0.498141,13,0,59,13,32,...,25,15,0,0,52,"['Basically there are many categories of "" Bes...",72.525622,10.477182,89.357222,0.860252
1,['salt is good for not dying in car crashes an...,0,216,4.014388,0.517986,17,5,93,18,67,...,28,8,2,3,54,['salt is good for not dying in car crashes an...,67.493766,10.975323,131.635424,1.227575


In [ ]:
df = df.drop(columns=['text_str'])

In [ ]:
label_encoder = LabelEncoder()
df['Label'] = label_encoder.fit_transform(df['Label'])

In [ ]:
df['Text'].fillna('', inplace=True)
df['Text'] = df['Text'].astype(str)

In [ ]:
X_text = df['Text'].values
Y = df['Label'].values

In [ ]:
# Tokenize text
maxlen = 100  # Adjust according to your text length
max_words = 10000  # Adjust according to your vocabulary size

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_text)
X_text = tokenizer.texts_to_sequences(X_text)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 92342 unique tokens.


In [ ]:
X_text = pad_sequences(X_text, maxlen=maxlen)

In [ ]:
X_text

array([[1357, 9784,  355, ...,   64,  755,   82],
       [  64,    2,   63, ...,    1,  145,  233],
       [ 592, 2076,   26, ...,    1, 3794,   26],
       ...,
       [   0,    0,    0, ...,   19,   55,   26],
       [ 325,  136,  123, ..., 2069, 1004,   26],
       [ 712,   27,   14, ...,    5,  165,  830]], dtype=int32)

In [ ]:
# Preprocess numerical data
numerical_data = df[['Vocab Size', 'Avg Word Length', 'Density', 'active',
                     'passive', 'noun', 'pron', 'verb', 'adj', 'adv', 'det', 'propn', 'part',
                     'intj', 'punct', 'Flesch Reading Ease', 'Gunning Fog Index',
                     'Perplexity', 'Burstness']].values

scaler = StandardScaler()
numerical_data = scaler.fit_transform(numerical_data)

In [ ]:
# Concatenate text and numerical data
X_text = np.asarray(X_text)
numerical_data = np.asarray(numerical_data)
Y = np.asarray(Y)
X = np.concatenate((X_text, numerical_data), axis=1)

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
X_text_train = X_train[:, :maxlen]
num_data_train = X_train[:, maxlen:]
X_text_val = X_val[:, :maxlen]
num_data_val = X_val[:, maxlen:]

In [ ]:
!pip install keras
from keras.utils import to_categorical
Y_train = to_categorical(Y_train, num_classes=2)
Y_val = to_categorical(Y_val, num_classes=2)

In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros
# Load GloVe word embeddings
embeddings_dictionary = {}
embedding_dim = 100  # Adjust according to the dimensions of your GloVe embeddings file

glove_file = open("/content/drive/MyDrive/Colab Notebooks/glove.6B.300d.txt")  # Update the path
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

vocab_size = len(tokenizer.word_index) + 1  ## total distinct words is the Vocabulary ##
word_index = tokenizer.word_index
num_words = min(max_words,len(word_index)+1)
embed_size = 300
embedding_matrix = zeros((num_words, embed_size)) ## has to be similar to glove dimension ##
for word, index in tokenizer.word_index.items():
    if index >= max_words:
        continue
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.04656   ,  0.21318001, -0.0074364 , ...,  0.0090611 ,
        -0.20988999,  0.053913  ],
       [-0.25756001, -0.057132  , -0.67189997, ..., -0.16043   ,
         0.046744  , -0.070621  ],
       ...,
       [ 0.020308  ,  0.042822  ,  0.23929   , ...,  0.093141  ,
         0.53148001, -0.039001  ],
       [-0.09712   , -0.38159001, -0.12734   , ..., -0.1268    ,
        -0.69660002,  0.39971   ],
       [-0.81866002,  0.58528   ,  0.014554  , ..., -0.20909999,
         0.087379  , -0.63577002]])

In [ ]:
input_text = Input(shape=(maxlen,), dtype='int32')
embedding_layer = Embedding(num_words, embed_size, weights=[embedding_matrix], trainable=False)(input_text)
lstm_layer = LSTM(128)(embedding_layer)

input_numerical = Input(shape=(numerical_data.shape[1],))
merged = Concatenate()([lstm_layer, input_numerical])
dense_layer = Dense(64, activation='relu')(merged)
dense_layer = Dropout(0.5)(dense_layer)
output = Dense(2, activation='softmax')(dense_layer)

model = Model(inputs=[input_text, input_numerical], outputs=output)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 100)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 100, 300)             3000000   ['input_1[0][0]']             
                                                                                                  
 lstm (LSTM)                 (None, 128)                  219648    ['embedding[0][0]']           
                                                                                                  
 input_2 (InputLayer)        [(None, 19)]                 0         []                            
                                                                                              

In [ ]:
model.fit([X_text_train, num_data_train], Y_train, epochs=10, batch_size=32, validation_data=([X_text_val, num_data_val], Y_val))

Epoch 1/10
1095/1095 [==============================] - 14s 10ms/step - loss: 0.1782 - accuracy: 0.9300 - val_loss: 0.0888 - val_accuracy: 0.9696
Epoch 2/10
1095/1095 [==============================] - 9s 9ms/step - loss: 0.0754 - accuracy: 0.9745 - val_loss: 0.0539 - val_accuracy: 0.9815
Epoch 3/10
1095/1095 [==============================] - 8s 8ms/step - loss: 0.0481 - accuracy: 0.9844 - val_loss: 0.0480 - val_accuracy: 0.9847
Epoch 4/10
1095/1095 [==============================] - 9s 9ms/step - loss: 0.0353 - accuracy: 0.9885 - val_loss: 0.0480 - val_accuracy: 0.9848
Epoch 5/10
1095/1095 [==============================] - 9s 8ms/step - loss: 0.0253 - accuracy: 0.9923 - val_loss: 0.0559 - val_accuracy: 0.9839
Epoch 6/10
1095/1095 [==============================] - 9s 8ms/step - loss: 0.0177 - accuracy: 0.9951 - val_loss: 0.0787 - val_accuracy: 0.9796
Epoch 7/10
1095/1095 [==============================] - 11s 10ms/step - loss: 0.0144 - accuracy: 0.9964 - val_loss: 0.0678 - val_accur

In [ ]:
# Save the model to a specific path
model.save('/content/drive/MyDrive/Colab Notebooks/RNN')

In [ ]:
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score

# Evaluate the model on the validation data
loss, accuracy = model.evaluate([X_text_val, num_data_val], Y_val, verbose=0)

# Predict the classes for the validation data
Y_pred = model.predict([X_text_val, num_data_val])
Y_pred_classes = np.argmax(Y_pred, axis=1)
Y_true = np.argmax(Y_val, axis=1)

# Calculate precision, recall, and F1 score
precision = precision_score(Y_true, Y_pred_classes)
recall = recall_score(Y_true, Y_pred_classes)
f1 = f1_score(Y_true, Y_pred_classes)

print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

# Generate and print the classification report
report = classification_report(Y_true, Y_pred_classes, target_names=['Human', 'Machine'])
print(report)

274/274 [==============================] - 3s 7ms/step
Accuracy: 0.98
Precision: 0.99
Recall: 0.97
F1 Score: 0.98
              precision    recall  f1-score   support

       Human       0.97      0.99      0.98      4405
     Machine       0.99      0.97      0.98      4351

    accuracy                           0.98      8756
   macro avg       0.98      0.98      0.98      8756
weighted avg       0.98      0.98      0.98      8756



In [ ]:
from sklearn.preprocessing import StandardScaler
from keras.models import load_model


In [ ]:
# Load the test dataset
test_df = pd.read_csv("/content/drive/MyDrive/Classroom/testdataset.csv")

In [ ]:
# **Load or create the training dataset**
# Assuming the training data is in a CSV file
train_df = pd.read_csv("/content/drive/MyDrive/Classroom/testdataset.csv")

In [ ]:
# Preprocess the test dataset
test_df['Text'].fillna('', inplace=True)
test_df['Text'] = test_df['Text'].astype(str)

feature_columns = ['Vocab Size', 'Avg Word Length', 'Density', 'active',
                   'passive', 'noun', 'pron', 'verb', 'adj', 'adv', 'det', 'propn',
                   'part', 'intj', 'punct', 'Flesch Reading Ease',
                   'Gunning Fog Index', 'Perplexity', 'Burstness']

scaler = StandardScaler()

In [ ]:
# Fit the scaler to your training data
scaler.fit(train_df[feature_columns])
test_df[feature_columns] = scaler.transform(test_df[feature_columns])

In [ ]:
# Tokenize the text data
maxlen = 100  # or the length you used during training
X_text_test = tokenizer.texts_to_sequences(test_df['Text'])
X_text_test = pad_sequences(X_text_test, maxlen=maxlen)
num_data_test = test_df[feature_columns].values

In [ ]:
# Load the trained RNN model
model_path ='/content/drive/MyDrive/Colab Notebooks/RNN'
model = load_model(model_path)

In [ ]:
# Evaluate the model on the test data
Y_test = test_df['Label'].values
Y_test = np.eye(2)[Y_test]
loss, accuracy = model.evaluate([X_text_test, num_data_test], Y_test, verbose=0)
print(f'Test Accuracy: {accuracy*100:.2f}%')


Test Accuracy: 96.03%


In [ ]:
# Predict the classes for the test data
Y_pred = model.predict([X_text_test, num_data_test])
Y_pred_classes = np.argmax(Y_pred, axis=1)
Y_true = np.argmax(Y_test, axis=1)


152/152 [==============================] - 1s 4ms/step


In [ ]:
# Generate and print the classification report
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')
from sklearn.metrics import classification_report
report = classification_report(Y_true, Y_pred_classes, target_names=['Class 0', 'Class 1'])
print(report)

Accuracy: 0.96
Precision: 0.99
Recall: 0.97
F1 Score: 0.98
              precision    recall  f1-score   support

     Class 0       0.94      0.99      0.96      2432
     Class 1       0.98      0.94      0.96      2432

    accuracy                           0.96      4864
   macro avg       0.96      0.96      0.96      4864
weighted avg       0.96      0.96      0.96      4864



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, classification_report
# Generate and plot the confusion matrix
cm = confusion_matrix(Y_true, Y_pred_classes)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Human', 'Machine'], yticklabels=['Human', 'Machine'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Compute ROC curve and AUC
fpr, tpr, _ = roc_curve(Y_true, Y_pred[:, 1])
roc_auc = roc_auc_score(Y_true, Y_pred[:, 1])

# Plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()
